# Importing Libraries

In [1]:
import nbimporter  # Notebook importer: enables importing modules defined in Jupyter notebooks—useful for structuring analysis pipelines interactively.
from Step1_processing import red_tags_extracted  # Import custom red‐tag detection function from your Step1_processing module.
from Step2_FHB_related_traits_extract import trait_output_red_tags, trait_output_without_red_tags # Import custom trait_output_red_tags, trait_output_without_red_tags function from your Step2_FHB_related_traits_extract module.

import numpy as np  
import pandas as pd  
from skimage import io, color, measure, img_as_ubyte, morphology  
from skimage.io import imread  
from skimage.draw import polygon, line  
from skimage.measure import label, regionprops  
from skimage.morphology import (
    square, disk,             
    closing, opening,          
    erosion, binary_dilation,   
    remove_small_objects        
)

from skimage import exposure, filters 
from skimage.util import img_as_ubyte  
import cv2 
from ultralytics import YOLO  # Ultralytics YOLO: state‐of‐the‐art object detection framework,.
import matplotlib.pyplot as plt  
from matplotlib import patches 
from PIL import Image  
import os  
from scipy.spatial.distance import euclidean, cdist  
from sklearn.preprocessing import StandardScaler  
import joblib  

In [2]:
def predict_disease_label(Area_value, Length_value, model_data):
    
    # Extract the standardized features (X_scaled), cluster labels, and the scaler used during training
    X_scaled, clusters, scaler = model_data['X_scaled'], model_data['clusters'], model_data['scaler']
    
    # Standardize the input data (CumSxArea_Region and CumSxLen_Region) to match the scale of the training data
    data = np.array([Area_value, Length_value]).reshape(1, -1)
    new_features_scaled = scaler.transform(data)

    # Calculate the Euclidean distance between the new data point and the existing scaled feature points
    # and find the closest cluster to the new data
    closest_cluster = np.argmin(cdist(new_features_scaled, X_scaled, metric='euclidean'))

    # Map the cluster number to a corresponding disease label (e.g., HR, MR, MS, HS)
    labels_map = {0: "HR", 3: "MR", 2: "MS", 1: "HS"}
    
    # Return the predicted disease label based on the closest cluster
    # If the cluster is not found in the labels map, return "Unknown"
    return labels_map.get(clusters[closest_cluster], "Unknown")

In [3]:
# Clustering model
model_data = joblib.load("Model_save/Resistance classfication model.joblib") 
model = YOLO("Model_save/Yolov8-WFD.pt") 

# Evaluating resistance level(With of red rags)

Single image

In [4]:
# Single Image Resistance Level Prediction Workflow
image_path = "Data/2.jpg"  # Path to the input image 

# 1. Feature Extraction (Red Mark Detection)
_, SxSpkPct_ROI, CumSxArea_ROI, AvgSxArea_Spk, CumSxLen_ROI, AvgSxLen_Spk, _, _ = trait_output_red_tags(image_path, model)

# 2. Disease Resistance Prediction (Model or Rule-Based)
predicted_labels = predict_disease_label(CumSxArea_ROI, CumSxLen_ROI, model_data)

# 3. Organize Data into DataFrame for Results

result_df = pd.DataFrame({
    "Image_name": [os.path.basename(image_path)],  
    "SxSpkPct_ROI": [SxSpkPct_ROI],  
    "CumSxArea_ROI": [CumSxArea_ROI],  
    "AvgSxArea_Spk": [AvgSxArea_Spk], 
    "CumSxLen_ROI": [CumSxLen_ROI],  
    "AvgSxLen_Spk": [AvgSxLen_Spk],  
    "Resistance_Level": [predicted_labels]  
})

# 4. Display Results
display(result_df)




image 1/1 E:\Evaluation platform-wheat FHB\Data\2.jpg: 640x352 5 scabs, 65 spikes, 148.0ms
Speed: 2.0ms preprocess, 148.0ms inference, 67.3ms postprocess per image at shape (1, 3, 640, 352)


,Image_name,SxSpkPct_ROI,CumSxArea_ROI,AvgSxArea_Spk,CumSxLen_ROI,AvgSxLen_Spk,Resistance_Level
0,2.jpg,0.119048,4.66492,0.856923,5.370165,0.86015,MR


Batch processing

In [5]:
# Folder containing the images for processing
input_folder = r"Data"  # Define the directory where the input images are stored.

# Initialize an empty list to collect results across all images
results_data = []  # This will hold data such as image name, extracted features, and predicted resistance levels.

# Iterate over all images in the input folder
for image_name in os.listdir(input_folder):  # Loop through each file in the specified directory
    image_path = os.path.join(input_folder, image_name)  # Get the full path to the image file
    
    # These traits can be extract.
    _, SxSpkPct_ROI, CumSxArea_ROI, AvgSxArea_Spk, CumSxLen_ROI, AvgSxLen_Spk, _, _ = trait_output_red_tags(image_path, model)
    
    # This prediction is based on a predefined model or rule set.
    predicted_labels = predict_disease_label(CumSxArea_ROI, CumSxLen_ROI, model_data)
    
    # Store the results for the current image
    results_data.append([image_name, SxSpkPct_ROI, CumSxArea_ROI, AvgSxArea_Spk, CumSxLen_ROI, AvgSxLen_Spk, predicted_labels])

# Each row in the DataFrame corresponds to an image with its associated metrics and prediction.
results_df = pd.DataFrame(results_data, columns=[
    "Image name", "SxSpkPct_ROI", "CumSxArea_ROI", "AvgSxArea_Spk", 
    "CumSxLen_ROI", "AvgSxLen_Spk", "Resistance level"
])

# The results will be saved to a specific directory for later use.
output_excel_path = r"Result\Results_with_red_tags.xlsx"  # Define the output file path for the Excel file
results_df.to_excel(output_excel_path, index=False)  # Save DataFrame to Excel without including row index

# Optionally, display a message confirming successful save
print(f"Results saved to {output_excel_path}")  





image 1/1 E:\Evaluation platform-wheat FHB\Data\1.jpg: 640x320 2 scabs, 103 spikes, 146.4ms
Speed: 1.8ms preprocess, 146.4ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 E:\Evaluation platform-wheat FHB\Data\2.jpg: 640x352 5 scabs, 65 spikes, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 352)

image 1/1 E:\Evaluation platform-wheat FHB\Data\3.jpg: 640x320 1 scab, 105 spikes, 8.1ms
Speed: 2.3ms preprocess, 8.1ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 E:\Evaluation platform-wheat FHB\Data\4.jpg: 640x320 8 scabs, 118 spikes, 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 4.0ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 E:\Evaluation platform-wheat FHB\Data\5.jpg: 640x288 9 scabs, 75 spikes, 131.8ms
Speed: 0.0ms preprocess, 131.8ms inference, 10.1ms postprocess per image at shape (1, 3, 640, 288)

image 1/1 E:\Evaluation platform-wheat FHB\Data\6.jpg: 640

# Evaluating resistance level(Without of red rags)

Single image

In [6]:
# Single Image Resistance Level Prediction Workflow
image_path="Data/2.jpg"# Path to the input image
# 1. Feature Extraction (Red Mark Detection)
SxSpkratio_Plot, CumSxArea_Plot, AvgSxArea_Spk, CumSxLen_Plot, AvgSxLen_Spk,_,_=trait_output_without_red_tags(image_path,model)
# 2. Disease Resistance Prediction (Model or Rule-Based)
predicted_labels = predict_disease_label(CumSxArea_Plot, CumSxLen_Plot, model_data)
# 3. Organize Data into DataFrame for Results
result_df = pd.DataFrame({
    "SxSpkratio_Plot": [SxSpkratio_Plot],
    "SxSpkPct_ROI": ["Nan"],
    "CumSxArea_Plot": [CumSxArea_Plot],
    "AvgSxArea_Spk": [AvgSxArea_Spk],
    "CumSxLen_Plot": [CumSxLen_Plot],
    "AvgSxLen_Spk": [AvgSxLen_Spk],
    "Resistance_Level":[predicted_labels]
})

# 4. Display Results
display(result_df)


image 1/1 E:\Evaluation platform-wheat FHB\Data\2.jpg: 640x352 5 scabs, 65 spikes, 32.0ms
Speed: 3.0ms preprocess, 32.0ms inference, 32.3ms postprocess per image at shape (1, 3, 640, 352)


,SxSpkratio_Plot,SxSpkPct_ROI,CumSxArea_Plot,AvgSxArea_Spk,CumSxLen_Plot,AvgSxLen_Spk,Resistance_Level
0,0.071429,Nan,4.284613,0.856923,4.300752,0.86015,MR


Batch processing

In [7]:
# Folder containing the images for processing
input_folder = r"Data"  # Define the directory where the input images are stored.

# Initialize an empty list to collect results across all images
results_data = []  # This will hold data such as image name, extracted features, and predicted resistance levels.
# Iterate over all images in the folder
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    # These traits can be extract.
    SxSpkratio_Plot, CumSxArea_Plot, AvgSxArea_Spk, CumSxLen_Plot, AvgSxLen_Spk,_,_=trait_output_without_red_tags(image_path,model)
    
    # This prediction is based on a predefined model or rule set.
    predicted_labels = predict_disease_label(CumSxArea_Plot, CumSxLen_Plot, model_data)
    # Store the results for the current image
    results_data.append([image_name,SxSpkratio_Plot, CumSxArea_Plot, AvgSxArea_Spk, CumSxLen_Plot, AvgSxLen_Spk, predicted_labels])
# Each row in the DataFrame corresponds to an image with its associated metrics and prediction.
results_df = pd.DataFrame(results_data, columns=[
    "Image name", "SxSpkratio_Plot", "CumSxArea_Plot", "AvgSxArea_Spk", 
    "CumSxLen_Plot", "AvgSxLen_Spk", "Resistance level"
])

# The results will be saved to a specific directory for later use.
output_excel_path = r"Result\Results_without_red_tags.xlsx"  # Define the output file path for the Excel file
results_df.to_excel(output_excel_path, index=False)  # Save DataFrame to Excel without including row index

# Optionally, display a message confirming successful save
print(f"Results saved to {output_excel_path}")  



image 1/1 E:\Evaluation platform-wheat FHB\Data\1.jpg: 640x320 2 scabs, 103 spikes, 47.6ms
Speed: 0.0ms preprocess, 47.6ms inference, 31.8ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 E:\Evaluation platform-wheat FHB\Data\2.jpg: 640x352 5 scabs, 65 spikes, 15.7ms
Speed: 0.0ms preprocess, 15.7ms inference, 15.8ms postprocess per image at shape (1, 3, 640, 352)

image 1/1 E:\Evaluation platform-wheat FHB\Data\3.jpg: 640x320 1 scab, 105 spikes, 11.5ms
Speed: 3.0ms preprocess, 11.5ms inference, 20.6ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 E:\Evaluation platform-wheat FHB\Data\4.jpg: 640x320 8 scabs, 118 spikes, 10.2ms
Speed: 0.0ms preprocess, 10.2ms inference, 20.6ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 E:\Evaluation platform-wheat FHB\Data\5.jpg: 640x288 9 scabs, 75 spikes, 20.1ms
Speed: 0.0ms preprocess, 20.1ms inference, 10.1ms postprocess per image at shape (1, 3, 640, 288)

image 1/1 E:\Evaluation platform-wheat FHB\Data\6.jp